# Modelling scheduling problem as constraint programming 

In this AIBT you learnt how to model a combinatorial optimisation problem using either Constraint Programming or Mixed Integer Linear programming paradigm. https://www.xoolive.org/optim4ai/

In this notebook you will be able to apply your modelling abilities to model RCPSP problem.
Contrary to your first course, a more scheduling oriented CP solver will be used. It is the [Ortools CPSAT solver](https://developers.google.com/optimization/cp/cp_solver)

<div class="alert alert-warning">
    <b>Questions</b> appear in yellow.
</div>

## Ortools basics

In [ ]:
from ortools.sat.python import cp_model

### Creating a CP Model : 

In [ ]:
model = cp_model.CpModel()

### Creating your first integer variables

In [ ]:
# Doc : 
model.NewIntVar??

In [ ]:
x = model.NewIntVar(0, 10, "x")
y = model.NewIntVar(0, 20, "y")

### Creating your first constraint

In [ ]:
model.Add(x+y<=10)

### Setting objective function

In [ ]:
model.Maximize(x+y)

### Solving

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model)
status_human = solver.StatusName(status)
print(status_human)

In [ ]:
value_x = solver.Value(x)
value_y = solver.Value(y)
print(value_x, value_y)

### Interval variables for scheduling

CP-SAT solver from Ortools has a big focus on scheduling problems. Therefore, it uses the concept of interval variables

In [ ]:
model.NewIntervalVar??

Here's a simple example where we have 1 task to schedule with a duration of 10 and we want to schedule it as late as possible considering the allowed range $[0, 20]$

In [ ]:
model = cp_model.CpModel()
start = model.NewIntVar(0, 20, "start")
end = model.NewIntVar(0, 20, "end")
duration = 10
task_var = model.NewIntervalVar(start, duration, end, name="task")
model.Maximize(start)

solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model)
status_human = solver.StatusName(status)
print(status_human)
print("Task scheduled between time : ", solver.Value(start), solver.Value(end))

### Global constraints on interval

#### No overlap constraint
The <b>NoOverlap</b> constraint takes as input a list of interval variables, and forbid any overlap between the intervals. It can be usefull for problem where a given list of task have to be done by the same machine or worker.

In [ ]:
# Look at the documentation in ortools library
model.AddNoOverlap?

#### Cumulative resource constraint
The <b>Cumulative</b> constraint insures that at any time, a set of interval consuming a given quantity of resource don't overconsume a given capacity.

In [ ]:
model.AddCumulative?

## Job shop problem

<div class="alert alert-warning">
<b>Problem #1: (medium)</b><br />

Simple job shop problem
</div>

<div class="alert alert-info">
The original problem is stated as follows:
<blockquote>
Each job consists of a sequence of tasks, which must be performed in a given order, and each task must be processed on a specific machine. For example, the job could be the manufacture of a single consumer item, such as an automobile. The problem is to schedule the tasks on the machines so as to minimize the length of the schedule—the time it takes for all the jobs to be completed.
There are several constraints for the job shop problem:
No task for a job can be started until the previous task for that job is completed.
A machine can only work on one task at a time.
A task, once started, must run to completion.
<br/><br/>
</blockquote>

</div>

## Object oriented placeholder
Let's define some structure to store the input data of a job shop problem.
A jobshop problem is defined by a number of machines, a list of jobs themselves composed of several Subjob to execute in order. 

In [ ]:
from typing import List
class Subjob:
    machine_id: int
    processing_time: int
    def __init__(self, machine_id, processing_time):
        self.machine_id = machine_id
        self.processing_time = processing_time
    def __str__(self):
        return f"machine and duration : {self.machine_id, self.processing_time}"
        
class JobShopProblem:
    n_jobs: int
    n_machines: int
    list_jobs: List[List[Subjob]]
    def __init__(self, list_jobs: List[List[Subjob]], n_jobs: int=None, n_machines: int=None):
        self.n_jobs = n_jobs
        self.n_machines = n_machines
        self.list_jobs = list_jobs
        if self.n_jobs is None:
            self.n_jobs = len(list_jobs)
        if self.n_machines is None:
            self.n_machines = len(set([y.machine_id for x in self.list_jobs
                                       for y in x]))
        # Store for each machine the list of subjob given as (index_job, index_subjob)
        self.job_per_machines = {i: [] for i in range(self.n_machines)}
        for k in range(self.n_jobs):
            for sub_k in range(len(list_jobs[k])):
                self.job_per_machines[list_jobs[k][sub_k].machine_id] += [(k, sub_k)]

### Example : 

In [ ]:
job_0 = [Subjob(machine_id=0, processing_time=3), Subjob(1, 2), Subjob(2, 2)]
job_1 = [Subjob(0, 2), Subjob(2, 1), Subjob(1, 4)]
job_2 = [Subjob(1, 4), Subjob(2, 3)]
example_jobshop = JobShopProblem(list_jobs=[job_0, job_1, job_2])
print(example_jobshop.job_per_machines)

### Solution encoding : 
we will choose how to encode a schedule solution in a specific object too.

In [ ]:
from typing import Tuple
class SolutionJobshop:
    def __init__(self, schedule: List[List[Tuple[int, int]]]):
        # For each job and subjob, start and end time given as tuple of int.
        self.schedule = schedule

### Checking of a solution
We can code a python function verifying if a solution is valid. That could help you debug your CP model afterwards.

In [ ]:
def check_solution(solution: SolutionJobshop, problem: JobShopProblem):
    if len(solution.schedule)!=problem.n_jobs:
        print("solution schedule should be same size as the problem")
        return False
    for k in range(problem.n_jobs):
        if len(solution.schedule[k])!=len(problem.list_jobs[k]):
            print(f"solution schedule for task n°{k} should be coherent with problem")
            return False
        for sub_k in range(len(solution.schedule[k])):
            if solution.schedule[k][sub_k][1]-solution.schedule[k][sub_k][0]!=problem.list_jobs[k][sub_k].processing_time:
                print(f"Duration of task should be coherent with problem")
                return False
            if sub_k>=1:
                if not (solution.schedule[k][sub_k][0]>=solution.schedule[k][sub_k-1][1]):
                    print(f"Precedence constraint between consecutive subtask not respected")
                    return False
    for machine in problem.job_per_machines:
        sorted_job = sorted([solution.schedule[x[0]][x[1]]
                             for x in problem.job_per_machines[machine]])
        for l in range(1, len(sorted_job)):
            if not (sorted_job[l][0]>=sorted_job[l-1][1]):
                print("Some task are overlaping in one machine")
                return False
    print("Constraint satisfied")
    return True

### Plotting of a solution

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon as pp
from shapely.geometry import Polygon
from matplotlib.font_manager import FontProperties
def plot_solution(solution: SolutionJobshop, problem: JobShopProblem):
    fig, ax = plt.subplots(1)
    patches = []
    for machine in problem.job_per_machines:
        for task in problem.job_per_machines[machine]:
            time_start, time_end = solution.schedule[task[0]][task[1]]
            polygon = Polygon(
                        [
                            (time_start, machine-0.2),
                            (time_end, machine-0.2),
                            (time_end, machine+0.2),
                            (time_start, machine+0.2),
                            (time_start, machine-0.2),
                        ]
                    )
            ax.annotate(str(task),
                xy=((3*time_start+time_end)/4, machine),
                font_properties=FontProperties(size=7, weight="bold"),
                verticalalignment="center",
                horizontalalignment="left",
                color="k",
                clip_on=True,
            )
            x, y = polygon.exterior.xy
            ax.plot(x, y, zorder=-1, color="b")
            patches.append(pp(xy=polygon.exterior.coords))
    p = PatchCollection(patches, cmap=matplotlib.colormaps.get_cmap("Blues"), alpha=0.4)
    ax.add_collection(p)
    ax.set_yticks(range(problem.n_machines))
    ax.set_yticklabels(
        tuple([f"machine {j}" for j in range(problem.n_machines)]), fontdict={"size": 5}
    )
    return fig, ax

### Manual solution computation
<div class="alert alert-warning">
<b>Manual solution</b><br />

Here you are asked to build yourself a solution, that pass the check solution and that you can plot.
</div>

In [ ]:
handcrafted_solution = SolutionJobshop([[[?, ?], [?, ?], [?, ?]], 
                                        [[?, ?], [?, ?], [?, ?]], 
                                        [[?, ?], [?, ?]]])
check = check_solution(solution=handcrafted_solution, problem=example_jobshop)
plot_solution(handcrafted_solution, example_jobshop)
print("Solution respect the jobshop constraints : ", check)

In [ ]:
# %load correction/nb2_jobshophandcrafted.py

### CP Modelling
Constraint programming powerness can help our scheduling tasks by providing good quality schedules fastly. In this section you will code the CP model of jobshop problem

<div class="alert alert-warning">
<b>CP solver implementation</b><br />

Job shop problem
</div>

In [ ]:
class SolverJobShop:
    def __init__(self, jobshop_problem: JobShopProblem):
        self.jobshop_problem = jobshop_problem
        self.model = cp_model.CpModel()
        self.variables = {}
    
    def init_model(max_time: int):
        # Write variables, constraints
        pass
        
    def solve() -> SolutionJobshop:
        self.init_model()
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 10
        status = solver.Solve(self.model)
        status_human = solver.StatusName(status)
        print(status_human)
        # Code the reconstruction of the
        return
        

In [ ]:
# %load correction/nb2_cpmodel_cell.py

### Test the solver on benchmark


In [ ]:
from jsplib_parser import create_jsplib_instance, JSPLIBInstance #, instance_names, instance_opti

model = create_jsplib_instance("abz5")
pb = model.jsplib_to_jobshop()
pb = JobShopProblem(list_jobs=[[Subjob(**x) for x in y] for y in pb])
print(pb.n_jobs)
print(pb.n_machines)
print(pb.list_jobs[0][0])
solver = SolverJobShop(jobshop_problem=pb)
solution = solver.solve(max_time=10000)
check_solution(solution, pb)
plot_solution(solution, pb)